# Modelling
## Introduction
In this section we develop models guided by insights from the exploratory data analysis. Our goal is to identify the factors that most strongly predict cyclist injury severity in San Francisco. We also estimate crash severity and crash count models to understand both the likelihood of severe outcomes when a crash occurs and the frequency of crashes across the network. This differs from studies such as Scarano et al. (2023), which use national datasets and more advanced modeling frameworks; our work applies similar count and severity models to San Francisco’s TIMS bicycle crash data. This is useful because a city-level analysis captures local patterns and street conditions that broader national studies cannot reflect.

## Crash Severity Model
The data consists of three crash severities. The outcome of this kind of statistical modelling is highly dependent on the proportion of data available for each crash severity.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

